**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
cat  ${FD_RES}/results/region/region_screened_selected.bed | expand -t 12

chr11       4000000     6600000     chr11:4091884-6505900
chr11       32800000    35000000    chr11:32869701-34870100
chr11       61700000    62000000    chr11:61787329-61898348
chr12       54290000    54310000    chr12:54300766-54301042
chr12       110000000   113000000   chr12:110726151-112325737
chr16       0           1300000     chr16:10001-1173100
chr19       12880000    12900000    chr19:12887110-12887237
chr2        59000000    62000000    chr2:59553301-61553700
chr20       55500000    59000000    chr20:56391301-58391700
chr3        128475000   128512000   chr3:128487571-128487937
chr4        55000000    56500000    chr4:55237590-56068531
chr5        86000000    91600000    chr5:87611207-89911163
chr6        108000000   111000000   chr6:108840570-109828800
chr6        133000000   138000000   chr6:134253831-136927585
chr8        122000000   136000000   chr8:126735901-128736550
chrX        47000000    51000000    chrX:47785501-49880650


In [3]:
ASSAY=A001_K562_WSTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
ls ${FD_INP}/*WGS*bed* | wc -l
ls ${FD_INP}/*WGS*bed* | xargs -n 1 basename

7
A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep3.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep4.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz


In [4]:
ASSAY=A001_K562_WSTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*WGS*bed*))
FP_INP=${FP_INPS[0]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP/WGS/SUBSET}

echo ${FP_INP}
echo
echo ${FN_INP}
echo
echo ${FN_OUT}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

A001_K562_WSTARRseq.Input.rep1.SUBSET.unstranded.bed.gz


## RUN

In [39]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/fragments_subset_wstarr.%a.txt \
    --array 0-6 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=A001_K562_WSTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*WGS*bed*))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

FD_OUT=${FD_RES}/results/${ASSAY}/fragment_count
FN_OUT=${FN_INP/WGS/SUBSET}
FP_OUT=${FD_OUT}/${FN_OUT}

FD_REG=${FD_RES}/results/region
FN_REG=region_screened_selected.bed
FP_REG=${FD_REG}/${FN_REG}

### show I/O file
echo "Input: " ${FP_INP}
echo
echo "show first few lines of the file"
zcat ${FP_INP} | head -5
echo
echo "Input: " ${FP_REG}
echo
echo "show lines of the file"
cat ${FP_REG}
echo

### annotation using intersect
bedtools intersect -a ${FP_INP} -b ${FP_REG} -wo | gzip -c > ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of the file:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29562357


## Check

In [40]:
cat ${FD_LOG}/fragments_subset_wstarr.0.txt

Hostname:           x3-05-1.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-12-23+22:56:34

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

show first few lines of the file
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.
chr1	17004	17500	chr1_17004_17500	1	.
chr1	17483	17898	chr1_17483_17898	1	.

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected.bed

show lines of the file
chr11	4000000	6600000	chr11:4091884-6505900
chr11	32800000	35000000	chr11:32869701-34870100
chr11	61700000	62000000	chr11:61787329-61898348
chr12	54290000	54310000	chr12:54300766-54301042
chr12	110000000	113000000	chr12:110726151-112325737
chr16	0	1300000	chr16:10001-1173100
chr19	12880000	12900000	chr19:12887110-12887237
chr2	59000000	62000000	chr2:59553301-61553700
chr20	55500000	590

In [41]:
cat ${FD_LOG}/fragments_subset_wstarr.2.txt

Hostname:           x3-01-1.genome.duke.edu
Slurm Array Index:  2
Time Stamp:         03-12-23+22:56:34

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep3.WGS.unstranded.bed.gz

show first few lines of the file
chr1	10247	10299	chr1_10247_10299	1	.
chr1	14684	15184	chr1_14684_15184	1	.
chr1	14690	15158	chr1_14690_15158	1	.
chr1	14933	15190	chr1_14933_15190	1	.
chr1	15143	15820	chr1_15143_15820	1	.

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected.bed

show lines of the file
chr11	4000000	6600000	chr11:4091884-6505900
chr11	32800000	35000000	chr11:32869701-34870100
chr11	61700000	62000000	chr11:61787329-61898348
chr12	54290000	54310000	chr12:54300766-54301042
chr12	110000000	113000000	chr12:110726151-112325737
chr16	0	1300000	chr16:10001-1173100
chr19	12880000	12900000	chr19:12887110-12887237
chr2	59000000	62000000	chr2:59553301-61553700
chr20	55500000	590

In [42]:
cat ${FD_LOG}/fragments_subset_wstarr.6.txt

Hostname:           x3-01-2.genome.duke.edu
Slurm Array Index:  6
Time Stamp:         03-12-23+22:56:34

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz

show first few lines of the file
chr1	10537	11067	chr1_10537_11067	1	.
chr1	14408	14799	chr1_14408_14799	1	.
chr1	14498	14937	chr1_14498_14937	1	.
chr1	14676	15184	chr1_14676_15184	1	.
chr1	14684	15162	chr1_14684_15162	1	.

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected.bed

show lines of the file
chr11	4000000	6600000	chr11:4091884-6505900
chr11	32800000	35000000	chr11:32869701-34870100
chr11	61700000	62000000	chr11:61787329-61898348
chr12	54290000	54310000	chr12:54300766-54301042
chr12	110000000	113000000	chr12:110726151-112325737
chr16	0	1300000	chr16:10001-1173100
chr19	12880000	12900000	chr19:12887110-12887237
chr2	59000000	62000000	chr2:59553301-61553700
chr20	55500000	59

In [11]:
FPATH=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz
zcat ${FPATH} | head -n 38522889 | tail -n 1


gzip: /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz: unexpected end of file
chr10	99423542	99423

In [13]:
FPATH=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz
zcat ${FPATH} | head -n 38522900 > test.txt


gzip: /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz: unexpected end of file


In [14]:
cat test.txt | tail -n 20

chr10	99423331	99423712	chr10_99423331_99423712	1	.
chr10	99423334	99423604	chr10_99423334_99423604	1	.
chr10	99423345	99423850	chr10_99423345_99423850	1	.
chr10	99423355	99423576	chr10_99423355_99423576	1	.
chr10	99423363	99423729	chr10_99423363_99423729	1	.
chr10	99423371	99423632	chr10_99423371_99423632	1	.
chr10	99423388	99423526	chr10_99423388_99423526	1	.
chr10	99423415	99423860	chr10_99423415_99423860	1	.
chr10	99423418	99423880	chr10_99423418_99423880	1	.
chr10	99423419	99423996	chr10_99423419_99423996	1	.
chr10	99423425	99423766	chr10_99423425_99423766	1	.
chr10	99423430	99423807	chr10_99423430_99423807	1	.
chr10	99423444	99423932	chr10_99423444_99423932	1	.
chr10	99423448	99423947	chr10_99423448_99423947	1	.
chr10	99423495	99423995	chr10_99423495_99423995	1	.
chr10	99423503	99423936	chr10_99423503_99423936	1	.
chr10	99423512	99424033	chr10_99423512_99424033	1	.
chr10	99423532	99423907	chr10_99423532_99423907	1	.
chr10	99423537	99423921	chr10_99423537_99423921	1	.
chr10	994235

In [19]:
FPATH=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz
zcat ${FPATH} | wc -l


gzip: /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz: unexpected end of file
38522888


In [25]:
FPATH=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/A001_K562_wstarrseq_fragments_wgs
ls ${FPATH}

A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz
A001-input-K562-rep2.masked.dedup.fragments.counts.txt.gz
A001-input-K562-rep3.masked.dedup.fragments.counts.txt.gz
A001-input-K562-rep4.masked.dedup.fragments.counts.txt.gz
A001-K562-rep1.masked.dedup.fragments.counts.txt.gz
A001-K562-rep2.masked.dedup.fragments.counts.txt.gz
A001-K562-rep3.masked.dedup.fragments.counts.txt.gz


In [26]:
FPATH=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/A001_K562_wstarrseq_fragments_wgs/A001-K562-rep3.masked.dedup.fragments.counts.txt.gz
zcat ${FPATH} | wc -l


gzip: /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/A001_K562_wstarrseq_fragments_wgs/A001-K562-rep3.masked.dedup.fragments.counts.txt.gz: unexpected end of file
38522888


In [27]:
FPATH=/data/reddylab/Alex/encode4_duke/data/starr_seq/fragments/A001-K562-rep3.masked.dedup.fragments.counts.txt.gz
zcat ${FPATH} | wc -l

bash: A001-K562-rep3.masked.dedup.fragments.counts.txt.gz: command not found
328185275


In [ ]:
ls /data/reddylab/Alex/encode4_duke/data/starr_seq/fragments

In [38]:
FDIRY=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/A001_K562_wstarrseq_fragments_wgs
FPATHS=($(ls ${FDIRY}/*gz))
for FPATH in ${FPATHS[@]}; do
    FNAME=$(basename ${FPATH})
    echo ${FNAME}
    zcat ${FPATH} | wc -l 
    echo
done

A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz
26908970

A001-input-K562-rep2.masked.dedup.fragments.counts.txt.gz
99899775

A001-input-K562-rep3.masked.dedup.fragments.counts.txt.gz
105623984

A001-input-K562-rep4.masked.dedup.fragments.counts.txt.gz
108635002

A001-K562-rep1.masked.dedup.fragments.counts.txt.gz
160349140

A001-K562-rep2.masked.dedup.fragments.counts.txt.gz
157326312

A001-K562-rep3.masked.dedup.fragments.counts.txt.gz

gzip: /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/A001_K562_wstarrseq_fragments_wgs/A001-K562-rep3.masked.dedup.fragments.counts.txt.gz: unexpected end of file
38522888



In [ ]:
FDIRY=/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/A001_K562_wstarrseq_fragments_wgs
FPATHS=($(ls ${FDIRY}/*gz))
for FPATH in ${FPATHS[@]}; do
    FNAME=$(basename ${FPATH})
    echo ${FNAME}
    zcat ${FPATH} | wc -l 
    echo
done